<h2> Generating Faces With A WGan </h2>

In [2]:
import torch
import torchvision
import os
import PIL
import pdb
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import make_grid
from tqdm.auto import tqdm
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt


In [3]:
def show(tensor,num=25,name=''):
    data=tensor.detach().cpu()
    grid=make_grid(data[:num],nrow=5).permute(1,2,0)
    plt.imshow(grid.clip(0,1))
    plt.show
    

In [4]:
##hyperparameters and general parameters
n_epochs=10000
batchsize=128
lr=1e-4
zdim=200
device='cuda'

cur_step=0
crit_cycles=5
gen_losses=[]
crit_losses=[]
show_step=35
save_step=35



In [8]:
#Generator model
#n=>Input image width/height (or feature map channels for later layers)
#stride=> Stride of the kernel in the convolutional layer
#padding=>Padding applied to the input borders in the convolutional layer (e.g., 'same' or 'valid')
#ks=> Kernel size: Size of the convolutional kernel (filter)
#nn.ConvTranspose2d :(n-1)*stride-2*padding+ks
#nn.Conv2d : (n+2*pad-ks)//stride+1
class Generator(nn.Module):
    def __init__(self,z_dim=64,d_dim=16):
        super (Generator,self).__init()
        self.z_dim=z_dim
        #we begin with a 1x1 image with z_dim number of channels (200)
        self.gen=nn.sequential(
            nn.ConvTranspose2d(z_dim,d_dim*32,kernel_size=4,stride=1,padding=0), #(((1-1)*1)-2*0)+4=4 so size 4x4 and (ch=200(z_dim),512(d_dim*32))
            nn.BatchNorm2d(d_dim*32),
            nn.ReLU(True),
            
            nn.ConvTranspose2d(d_dim*32,d_dim*16,kernel_size=4,stride=2,padding=1),#(((4-1)*2)-2*1)+4=8 so size 8x8 and (ch=512(d_dim*32),256(d_dim*16))
            nn.BatchNorm2d(d_dim*16),
            nn.ReLU(True),
            
            nn.ConvTranspose2d(d_dim*8,d_dim*4,kernel_size=4,stride=2,padding=1),#(((16-1)*2)-2*1)+4=32 so size 32x32 and (ch=128(d_dim*8),64(d_dim*4))
            nn.BatchNorm2d(d_dim*4),
            nn.ReLU(True),

            nn.ConvTranspose2d(d_dim*4,d_dim*2,kernel_size=4,stride=2,padding=1),#(((32-1)*2)-2*1)+4=64 so size 64x64 and (ch=64(d_dim*4),32(d_dim*2))
            nn.BatchNorm2d(d_dim*2),
            nn.ReLU(True),

            nn.ConvTranspose2d(d_dim*2,3,kernel_size=4,stride=2,padding=1),#(((64-1)*2)-2*1)+4=128 so size 128x128 and (ch=32(d_dim*2),3)
            nn.Tanh() #Produce result in the range from -1 to 1
                       
        )


    def forward(self,noise):
        x=noise.view(len(noise),self.z_dim,1,1) #128x200x1x1
        return self.gen(x)



def gen_noise(num,z_dim,device='cuda'):
    return torch.randn(num,z_dim,device=device)#128x200
    
    

In [16]:
#Critic model
#conv2d: in_channels, out_channels,kernel_size,stride=1,padding=0
#new width and height: #(n+2*pad-ks)//stride+1
class Critic(nn.Module):
    def __init__(self,d_dim=16):
        super(Critic,self).__init__()
        self.crit=nn.Sequential(
        #conv2d: in_channels, out_channels,kernel_size,stride=1,padding=0
        #new width and height: #(n+2*pad-ks)//stride+1
        nn.Conv2d(3,d_dim,kernel_size=4,stride=2,padding=1), #(128+2*1-4)//2+1=64 so new size 64x64 and ch:(3,16(d_dim))
        nn.InstanceNorm2d(d_dim),
        nn.LeakyReLU(0.2),

        nn.Conv2d(d_dim,d_dim*2,kernel_size=4,stride=2,padding=1), #(64+2*1-4)//2+1=32 so new size 32x32 and ch:(16(d_dim),32(d_dim*2))
        nn.InstanceNorm2d(d_dim*2),
        nn.LeakyReLU(0.2),

        nn.Conv2d(d_dim*2,d_dim*4,kernel_size=4,stride=2,padding=1), #(32+2*1-4)//2+1=16 so new size 16x16 and ch:(32(d_dim*2),64(d_dim*4))
        nn.InstanceNorm2d(d_dim*4),
        nn.LeakyReLU(0.2),

        nn.Conv2d(d_dim*4,d_dim*8,kernel_size=4,stride=2,padding=1), #(16+2*1-4)//2+1=8 so new size 8x8 and ch:(64(d_dim*4),128(d_dim*8))
        nn.InstanceNorm2d(d_dim*8),
        nn.LeakyReLU(0.2),
        
        nn.Conv2d(d_dim*8,d_dim*16,kernel_size=4,stride=2,padding=1), #(8+2*1-4)//2+1=4 so new size 4x4 and ch:(128(d_dim*8),256(d_dim*16))
        nn.InstanceNorm2d(d_dim*16),
        nn.LeakyReLU(0.2),

        nn.Conv2d(d_dim*16,1,kernel_size=4,stride=1,padding=0), #(4+2*1-4)//2+0=1 so new size 1x1 and ch:(256(d_dim*16),1)
       )
    def forward(self,image):
       #image:128x3x128x128
        crit_pred=self.crit(image) #128x1x1x1   batch size x channel x widht x height
        return crit_pred.view(len(crit_pred),-1) #128x1